In [1]:
import torch

from deep_reorder.models import qwen25

import error: No module named 'triton'


In [2]:
model = qwen25.load_model(weights_path="../build/Qwen2_5-3B.safetensors", device='mps')

In [3]:
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    return param_size / 1024 ** 3  # Size in GB

In [4]:
get_model_size(model)

5.748008728027344

In [5]:
tokenizer = qwen25.load_tokenizer('../build/vocab.json', '../build/merges.txt', '../build/tokenizer.json')

In [6]:
params = qwen25.Qwen25GeneratorParams(max_length=15)

In [7]:
generator = qwen25.Qwen25Generator(model, tokenizer, params, device='mps')

In [8]:
prompt = "Who was the first president of the United States of"

In [9]:
generator.generate(prompt)

'Who was the first president of the United States of precisely precisely precisely precisely precisely'

In [20]:
input_ids = tokenizer.encode(prompt, add_eos=False)
input_ids = torch.tensor(input_ids).unsqueeze(0).to('mps')
input_ids

tensor([[15191,   572,   279,  1156,  4767,   315,   279,  3639,  4180,   315]],
       device='mps:0')

In [17]:
model(input_ids).shape

torch.Size([1, 10, 151936])

In [18]:
outputs = model(input_ids)

In [25]:
next_token_logits = outputs[:, -1, :].float()
next_token_logits = next_token_logits / params.temperature
next_token_logits

tensor([[-105.0000, -139.2857,    1.0547,  ...,  -53.5714,    9.2411,
            3.3482]], device='mps:0')

In [21]:
input_ids.shape

torch.Size([1, 10])

In [17]:
for key in model.state_dict().keys():
    print(f"{key:<50} | {model.state_dict()[key].dtype} | {model.state_dict()[key].shape}")

tok_embeddings.weight                              | torch.bfloat16 | torch.Size([151936, 2048])
layers.0.attn.q_proj.weight                        | torch.bfloat16 | torch.Size([2048, 2048])
layers.0.attn.q_proj.bias                          | torch.bfloat16 | torch.Size([2048])
layers.0.attn.k_proj.weight                        | torch.bfloat16 | torch.Size([256, 2048])
layers.0.attn.k_proj.bias                          | torch.bfloat16 | torch.Size([256])
layers.0.attn.v_proj.weight                        | torch.bfloat16 | torch.Size([256, 2048])
layers.0.attn.v_proj.bias                          | torch.bfloat16 | torch.Size([256])
layers.0.attn.output_proj.weight                   | torch.bfloat16 | torch.Size([2048, 2048])
layers.0.mlp.w1.weight                             | torch.bfloat16 | torch.Size([11008, 2048])
layers.0.mlp.w2.weight                             | torch.bfloat16 | torch.Size([2048, 11008])
layers.0.mlp.w3.weight                             | torch.bfloat16 